# 🎨 Exploração de Prompts - Pastagens Brasileiras

Este notebook permite explorar e testar o sistema de prompt engineering especializado em pastagens brasileiras.

**Funcionalidades:**
- ✅ Teste de prompts específicos por bioma
- ✅ Variações sazonais automáticas
- ✅ Análise de qualidade de imagens geradas
- ✅ Comparação visual entre diferentes configurações
- ✅ Exportação de prompts para uso posterior

## 📦 1. Importações e Setup

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Adicionar projeto ao path
if '/content/brazilian-pasture-synthesis' not in sys.path:
    sys.path.append('/content/brazilian-pasture-synthesis')
    
# Mudança para diretório do projeto se necessário
if not Path('src').exists():
    os.chdir('/content/brazilian-pasture-synthesis')

# Importações principais
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
import yaml
from tqdm.auto import tqdm
from dataclasses import asdict
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

# Importações do projeto
from src.diffusion.pipeline_manager import PipelineManager
from src.diffusion.prompt_engine import (
    PromptEngine, PastureConfig, Biome, Season, PastureQuality
)
from src.diffusion.image_postprocess import ImagePostProcessor, ProcessingConfig
from src.dataset.quality_metrics import QualityMetrics

# Configuração matplotlib
plt.style.use('default')
%matplotlib inline

print("✅ Importações concluídas!")
print(f"🖥️ Device disponível: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 🔧 2. Inicialização dos Componentes

In [ ]:
print("🔧 Inicializando componentes...")

# Inicializar pipeline manager
pipeline_manager = PipelineManager(
    model_name="stabilityai/stable-diffusion-xl-base-1.0",
    cache_dir="/content/model_cache",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Inicializar prompt engine
prompt_engine = PromptEngine(config_dir="configs/prompts")

# Inicializar pós-processador
post_processor = ImagePostProcessor()

# Inicializar avaliador de qualidade
quality_metrics = QualityMetrics()

print("✅ Componentes inicializados!")
print("⏳ Pipeline será carregado na primeira geração...")

## 🎛️ 3. Explorador Interativo de Prompts

In [ ]:
# Widgets para exploração interativa
biome_widget = widgets.Dropdown(
    options=[('Cerrado', 'cerrado'), ('Mata Atlântica', 'mata_atlantica'), ('Pampa', 'pampa')],
    value='cerrado',
    description='Bioma:'
)

season_widget = widgets.Dropdown(
    options=[('Estação Seca', 'seca'), ('Estação Chuvosa', 'chuvas'), ('Transição', 'transicao')],
    value='chuvas',
    description='Estação:'
)

quality_widget = widgets.Dropdown(
    options=[('Boa', 'boa'), ('Moderada', 'moderada'), ('Degradada', 'degradada')],
    value='moderada',
    description='Qualidade:'
)

coverage_widget = widgets.IntSlider(
    value=70,
    min=20,
    max=95,
    step=5,
    description='Cobertura (%)'
)

invasive_widget = widgets.SelectMultiple(
    options=[('Capim Gordura', 'capim_gordura'), ('Carqueja', 'carqueja'), 
             ('Samambaia', 'samambaia'), ('Cupinzeiro', 'cupinzeiro')],
    description='Invasoras:'
)

# Botões de ação
generate_btn = widgets.Button(
    description='🎨 Gerar Prompt',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

generate_image_btn = widgets.Button(
    description='🖼️ Gerar Imagem',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

# Widget de saída
output_widget = widgets.Output()

# Layout
config_box = widgets.VBox([
    widgets.HTML("<h3>🔧 Configuração da Pastagem</h3>"),
    biome_widget,
    season_widget, 
    quality_widget,
    coverage_widget,
    invasive_widget
])

button_box = widgets.HBox([generate_btn, generate_image_btn])

# Variáveis globais para armazenar resultados
current_prompt = ""
current_config = None
generated_image = None

def generate_prompt_callback(btn):
    global current_prompt, current_config
    
    with output_widget:
        clear_output()
        print("🎨 Gerando prompt...")
        
        # Criar configuração
        current_config = PastureConfig(
            biome=Biome(biome_widget.value),
            season=Season(season_widget.value),
            quality=PastureQuality(quality_widget.value),
            invasive_species=list(invasive_widget.value),
            grass_coverage=coverage_widget.value,
            soil_exposure=100 - coverage_widget.value
        )
        
        # Gerar prompt
        positive_prompt, negative_prompt = prompt_engine.generate_prompt(
            current_config, variation=True
        )
        current_prompt = positive_prompt
        
        # Exibir resultados
        print("✅ Prompt gerado com sucesso!")
        print("\n" + "="*60)
        print("🔹 CONFIGURAÇÃO:")
        print(f"   Bioma: {current_config.biome.value.title()}")
        print(f"   Estação: {current_config.season.value.title()}")
        print(f"   Qualidade: {current_config.quality.value.title()}")
        print(f"   Cobertura: {current_config.grass_coverage}%")
        print(f"   Invasoras: {', '.join(current_config.invasive_species)}")
        
        print("\n🔹 PROMPT POSITIVO:")
        print(f'   "{positive_prompt}"')
        
        print("\n🔹 PROMPT NEGATIVO:")
        print(f'   "{negative_prompt}"')
        print("="*60)
        
        # Habilitar botão de geração de imagem
        generate_image_btn.disabled = False

def generate_image_callback(btn):
    global generated_image
    
    if not current_prompt:
        with output_widget:
            print("❌ Gere um prompt primeiro!")
        return
        
    with output_widget:
        print("\n🖼️ Gerando imagem...")
        print("⏳ Isso pode demorar alguns minutos na primeira vez...")
        
        try:
            # Carregar pipeline se necessário
            if pipeline_manager.base_pipeline is None:
                print("📥 Carregando modelo Stable Diffusion...")
                pipeline_manager.load_base_pipeline()
                
            # Gerar imagem
            start_time = time.time()
            result = pipeline_manager.generate_image(
                prompt=current_prompt,
                negative_prompt=prompt_engine._generate_negative_prompt(current_config),
                num_inference_steps=25,  # Reduzido para teste rápido
                guidance_scale=8.0,
                width=1024,
                height=1024,
                seed=42  # Seed fixa para reprodutibilidade
            )
            
            generation_time = time.time() - start_time
            generated_image = result['image']
            
            # Pós-processamento
            print("🔧 Aplicando pós-processamento...")
            processed_image, quality_report = post_processor.process_image(
                generated_image,
                season=current_config.season.value,
                biome=current_config.biome.value
            )
            
            # Exibir resultado
            print(f"✅ Imagem gerada em {generation_time:.1f}s")
            print(f"📊 Score de qualidade: {quality_report.overall_score:.3f}")
            
            # Mostrar imagem
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
            
            ax1.imshow(generated_image)
            ax1.set_title("Imagem Original", fontsize=14)
            ax1.axis('off')
            
            ax2.imshow(processed_image)
            ax2.set_title(f"Pós-processada (Score: {quality_report.overall_score:.3f})", fontsize=14)
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Mostrar métricas detalhadas
            print("\n📊 MÉTRICAS DE QUALIDADE:")
            print(f"   Qualidade Técnica: {quality_report.technical_quality:.3f}")
            print(f"   Realismo Agronômico: {quality_report.agricultural_realism:.3f}")
            print(f"   Consistência Sazonal: {quality_report.seasonal_consistency:.3f}")
            
            if quality_report.recommendations:
                print("\n💡 RECOMENDAÇÕES:")
                for rec in quality_report.recommendations[:3]:  # Mostrar apenas as 3 principais
                    print(f"   • {rec}")
                    
        except Exception as e:
            print(f"❌ Erro na geração: {e}")
            print("💡 Tente reiniciar o runtime se persistir")

# Conectar callbacks
generate_btn.on_click(generate_prompt_callback)
generate_image_btn.on_click(generate_image_callback)

# Desabilitar botão de imagem inicialmente
generate_image_btn.disabled = True

# Exibir interface
display(widgets.VBox([
    widgets.HTML("<h2>🎨 Explorador de Prompts Interativo</h2>"),
    config_box,
    button_box,
    output_widget
]))

## 🧪 4. Teste em Lote - Comparação de Biomas

In [ ]:
def compare_biomes_prompts(season='chuvas', quality='moderada'):
    """
    Compara prompts gerados para diferentes biomas
    """
    print(f"🔍 Comparando prompts para estação: {season}, qualidade: {quality}")
    print("=" * 80)
    
    biomes = ['cerrado', 'mata_atlantica', 'pampa']
    biome_names = {'cerrado': 'Cerrado', 'mata_atlantica': 'Mata Atlântica', 'pampa': 'Pampa'}
    
    for biome in biomes:
        print(f"\n🌿 {biome_names[biome].upper()}")
        print("-" * 40)
        
        # Criar configuração
        config = PastureConfig(
            biome=Biome(biome),
            season=Season(season),
            quality=PastureQuality(quality),
            invasive_species=['capim_gordura'] if quality != 'boa' else [],
            grass_coverage=75 if quality == 'boa' else 60,
            soil_exposure=25 if quality == 'boa' else 40
        )
        
        # Gerar prompt
        positive, negative = prompt_engine.generate_prompt(config, variation=False)
        
        print(f"Prompt: {positive[:100]}...")
        print(f"Características: {config.invasive_species}, {config.grass_coverage}% cobertura")

# Executar comparação
compare_biomes_prompts('chuvas', 'moderada')
print("\n" + "="*80)
compare_biomes_prompts('seca', 'degradada')

## 🌦️ 5. Análise de Variações Sazonais

In [ ]:
def analyze_seasonal_variations(biome='cerrado'):
    """
    Analisa como os prompts variam entre estações para um bioma específico
    """
    print(f"🌦️ Análise sazonal para: {biome.title()}")
    print("=" * 60)
    
    seasons = ['seca', 'chuvas', 'transicao']
    season_names = {
        'seca': 'Estação Seca',
        'chuvas': 'Estação Chuvosa', 
        'transicao': 'Transição'
    }
    
    seasonal_data = []
    
    for season in seasons:
        print(f"\n☀️ {season_names[season]}")
        print("-" * 30)
        
        # Gerar múltiplas configurações para análise
        configs = []
        for quality in ['boa', 'moderada', 'degradada']:
            config = PastureConfig(
                biome=Biome(biome),
                season=Season(season),
                quality=PastureQuality(quality),
                invasive_species=[] if quality == 'boa' else ['capim_gordura'],
                grass_coverage=85 if quality == 'boa' else (65 if quality == 'moderada' else 40),
                soil_exposure=15 if quality == 'boa' else (35 if quality == 'moderada' else 60)
            )
            configs.append(config)
        
        # Analisar prompts gerados
        prompts = []
        for config in configs:
            positive, _ = prompt_engine.generate_prompt(config, variation=False)
            prompts.append(positive)
        
        # Extrair palavras-chave características
        common_words = set()
        for prompt in prompts:
            words = prompt.lower().split()
            for word in words:
                if len(word) > 4 and word.isalpha():
                    common_words.add(word)
        
        # Filtrar palavras relevantes para estação
        seasonal_keywords = {
            'seca': ['golden', 'yellow', 'brown', 'dried', 'sparse', 'exposed', 'harsh', 'intense'],
            'chuvas': ['green', 'lush', 'vibrant', 'dense', 'vigorous', 'humid', 'diffuse'],
            'transicao': ['mixed', 'variable', 'transition', 'changing', 'moderate']
        }
        
        found_keywords = [w for w in common_words if w in seasonal_keywords[season]]
        
        print(f"Palavras-chave típicas: {', '.join(found_keywords[:5])}")
        print(f"Cobertura média: {np.mean([c.grass_coverage for c in configs]):.1f}%")
        
        seasonal_data.append({
            'season': season,
            'keywords': found_keywords,
            'avg_coverage': np.mean([c.grass_coverage for c in configs]),
            'configs': configs
        })
    
    return seasonal_data

# Executar análise
cerrado_analysis = analyze_seasonal_variations('cerrado')
print("\n" + "="*60)
mata_analysis = analyze_seasonal_variations('mata_atlantica')

## 🚀 6. Geração em Lote para Teste

In [ ]:
def generate_test_batch(num_samples=6, fast_mode=True):
    """
    Gera lote de imagens para teste rápido dos prompts
    """
    print(f"🚀 Gerando lote de {num_samples} amostras de teste...")
    
    if pipeline_manager.base_pipeline is None:
        print("📥 Carregando pipeline...")
        pipeline_manager.load_base_pipeline()
    
    # Configurações de teste diversificadas
    test_configs = [
        # Cerrado variações
        PastureConfig(Biome.CERRADO, Season.CHUVAS, PastureQuality.BOA, [], 85, 15),
        PastureConfig(Biome.CERRADO, Season.SECA, PastureQuality.DEGRADADA, ['capim_gordura'], 35, 65),
        
        # Mata Atlântica variações  
        PastureConfig(Biome.MATA_ATLANTICA, Season.CHUVAS, PastureQuality.MODERADA, ['samambaia'], 65, 35),
        PastureConfig(Biome.MATA_ATLANTICA, Season.TRANSICAO, PastureQuality.BOA, [], 80, 20),
        
        # Pampa variações
        PastureConfig(Biome.PAMPA, Season.SECA, PastureQuality.MODERADA, ['carqueja'], 55, 45),
        PastureConfig(Biome.PAMPA, Season.CHUVAS, PastureQuality.BOA, [], 90, 10)
    ]
    
    results = []
    
    for i, config in enumerate(test_configs[:num_samples]):
        print(f"\n🎨 Gerando amostra {i+1}/{num_samples}...")
        print(f"   {config.biome.value} | {config.season.value} | {config.quality.value}")
        
        try:
            # Gerar prompt
            positive, negative = prompt_engine.generate_prompt(config, variation=True)
            
            # Parâmetros otimizados para teste rápido
            generation_params = {
                'prompt': positive,
                'negative_prompt': negative,
                'num_inference_steps': 15 if fast_mode else 25,
                'guidance_scale': 7.5,
                'width': 512 if fast_mode else 1024,
                'height': 512 if fast_mode else 1024,
                'seed': 42 + i  # Seeds diferentes para variedade
            }
            
            # Gerar imagem
            start_time = time.time()
            result = pipeline_manager.generate_image(**generation_params)
            generation_time = time.time() - start_time
            
            # Pós-processamento simples
            processed_image, quality_report = post_processor.process_image(
                result['image'],
                season=config.season.value,
                biome=config.biome.value
            )
            
            results.append({
                'config': config,
                'image': processed_image,
                'prompt': positive,
                'quality_score': quality_report.overall_score,
                'generation_time': generation_time
            })
            
            print(f"   ✅ Gerado em {generation_time:.1f}s | Score: {quality_report.overall_score:.3f}")
            
        except Exception as e:
            print(f"   ❌ Erro: {e}")
            continue
    
    # Exibir resultados em grid
    if results:
        print(f"\n🖼️ Exibindo {len(results)} imagens geradas...")
        
        cols = 3
        rows = (len(results) + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        if rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, result in enumerate(results):
            row, col = divmod(i, cols)
            
            axes[row, col].imshow(result['image'])
            
            # Título com informações
            config = result['config']
            title = f"{config.biome.value.title()} - {config.season.value.title()}\n"
            title += f"Qualidade: {config.quality.value} | Score: {result['quality_score']:.3f}"
            
            axes[row, col].set_title(title, fontsize=10)
            axes[row, col].axis('off')
        
        # Remover subplots vazios
        for i in range(len(results), rows * cols):
            row, col = divmod(i, cols)
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Estatísticas
        avg_quality = np.mean([r['quality_score'] for r in results])
        avg_time = np.mean([r['generation_time'] for r in results])
        
        print(f"\n📊 ESTATÍSTICAS DO LOTE:")
        print(f"   Qualidade média: {avg_quality:.3f}")
        print(f"   Tempo médio: {avg_time:.1f}s")
        print(f"   Taxa de sucesso: {len(results)}/{num_samples} ({len(results)/num_samples*100:.0f}%)")
        
    return results

# Widget para execução
batch_size_widget = widgets.IntSlider(
    value=6,
    min=3,
    max=12,
    description='Amostras:'
)

fast_mode_widget = widgets.Checkbox(
    value=True,
    description='Modo rápido (512px, 15 steps)'
)

batch_btn = widgets.Button(
    description='🚀 Gerar Lote',
    button_style='warning'
)

batch_output = widgets.Output()

def batch_callback(btn):
    with batch_output:
        clear_output()
        generate_test_batch(batch_size_widget.value, fast_mode_widget.value)

batch_btn.on_click(batch_callback)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Geração em Lote</h3>"),
    batch_size_widget,
    fast_mode_widget,
    batch_btn,
    batch_output
]))

## 💾 7. Exportação de Prompts

In [ ]:
def export_prompt_collection(num_prompts=50, filename="brazilian_pasture_prompts.json"):
    """
    Exporta coleção diversificada de prompts para uso posterior
    """
    print(f"💾 Gerando coleção de {num_prompts} prompts...")
    
    # Gerar configurações diversificadas
    prompt_collection = []
    
    for i in tqdm(range(num_prompts), desc="Gerando prompts"):
        # Configuração aleatória balanceada
        biome = np.random.choice(list(Biome))
        season = np.random.choice(list(Season))
        quality = np.random.choice(list(PastureQuality))
        
        # Invasoras baseadas na qualidade
        invasive_options = ['capim_gordura', 'carqueja', 'samambaia', 'cupinzeiro']
        if quality == PastureQuality.BOA:
            invasives = []
        elif quality == PastureQuality.MODERADA:
            invasives = np.random.choice(invasive_options, 
                                       size=np.random.randint(0, 2), 
                                       replace=False).tolist()
        else:  # DEGRADADA
            invasives = np.random.choice(invasive_options,
                                       size=np.random.randint(1, 3),
                                       replace=False).tolist()
        
        # Cobertura baseada na qualidade e estação
        if quality == PastureQuality.BOA:
            base_coverage = np.random.randint(75, 95)
        elif quality == PastureQuality.MODERADA:
            base_coverage = np.random.randint(50, 80)
        else:
            base_coverage = np.random.randint(20, 55)
            
        # Ajuste sazonal
        if season == Season.SECA:
            coverage = max(15, base_coverage - np.random.randint(10, 20))
        elif season == Season.CHUVAS:
            coverage = min(95, base_coverage + np.random.randint(0, 15))
        else:
            coverage = base_coverage
        
        # Criar configuração
        config = PastureConfig(
            biome=biome,
            season=season,
            quality=quality,
            invasive_species=invasives,
            grass_coverage=coverage,
            soil_exposure=100 - coverage
        )
        
        # Gerar prompts
        positive, negative = prompt_engine.generate_prompt(config, variation=True)
        
        # Adicionar à coleção
        prompt_data = {
            'id': i,
            'config': asdict(config),
            'positive_prompt': positive,
            'negative_prompt': negative,
            'generation_params': {
                'biome': config.biome.value,
                'season': config.season.value,
                'quality': config.quality.value,
                'coverage': config.grass_coverage,
                'invasives': config.invasive_species
            }
        }
        
        prompt_collection.append(prompt_data)
    
    # Estatísticas da coleção
    stats = {
        'total_prompts': len(prompt_collection),
        'biome_distribution': {},
        'season_distribution': {},
        'quality_distribution': {},
        'avg_coverage': np.mean([p['config']['grass_coverage'] for p in prompt_collection]),
        'generated_at': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Calcular distribuições
    for dist_key, enum_class in [('biome_distribution', Biome), 
                                 ('season_distribution', Season),
                                 ('quality_distribution', PastureQuality)]:
        for enum_val in enum_class:
            count = sum(1 for p in prompt_collection 
                       if p['config'][dist_key.split('_')[0]] == enum_val.value)
            stats[dist_key][enum_val.value] = count
    
    # Salvar coleção
    export_data = {
        'metadata': {
            'description': 'Brazilian Pasture Prompt Collection',
            'version': '1.0',
            'model_optimized_for': 'stable-diffusion-xl-base-1.0'
        },
        'statistics': stats,
        'prompts': prompt_collection
    }
    
    output_path = f"outputs/{filename}"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ Coleção exportada para: {output_path}")
    print(f"📊 Estatísticas:")
    print(f"   Total: {stats['total_prompts']} prompts")
    print(f"   Cobertura média: {stats['avg_coverage']:.1f}%")
    print(f"   Biomas: {dict(stats['biome_distribution'])}")
    print(f"   Estações: {dict(stats['season_distribution'])}")
    print(f"   Qualidades: {dict(stats['quality_distribution'])}")
    
    return output_path

# Interface de exportação
export_size_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=200,
    step=10,
    description='Quantidade:'
)

export_filename_widget = widgets.Text(
    value="brazilian_pasture_prompts.json",
    description='Nome do arquivo:'
)

export_btn = widgets.Button(
    description='💾 Exportar Prompts',
    button_style='success'
)

export_output = widgets.Output()

def export_callback(btn):
    with export_output:
        clear_output()
        export_prompt_collection(export_size_widget.value, export_filename_widget.value)

export_btn.on_click(export_callback)

display(widgets.VBox([
    widgets.HTML("<h3>💾 Exportação de Prompts</h3>"),
    export_size_widget,
    export_filename_widget,
    export_btn,
    export_output
]))

## 🧹 8. Limpeza de Memória

In [ ]:
def cleanup_memory():
    """Limpa memória GPU e cache"""
    global pipeline_manager, generated_image
    
    print("🧹 Limpando memória...")
    
    # Limpar variáveis globais
    generated_image = None
    
    # Descarregar modelos
    if pipeline_manager:
        pipeline_manager.unload_models()
    
    # Limpar cache GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ Cache GPU limpo")
    
    # Informações de memória
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"📊 Memória GPU: {allocated:.2f}GB alocada, {reserved:.2f}GB reservada")
    
    print("✅ Limpeza concluída!")

cleanup_btn = widgets.Button(
    description='🧹 Limpar Memória',
    button_style='danger'
)

cleanup_output = widgets.Output()

def cleanup_callback(btn):
    with cleanup_output:
        clear_output()
        cleanup_memory()

cleanup_btn.on_click(cleanup_callback)

display(widgets.VBox([
    widgets.HTML("<h3>🧹 Gerenciamento de Memória</h3>"),
    widgets.HTML("<p>Use para liberar memória GPU quando necessário</p>"),
    cleanup_btn,
    cleanup_output
]))

## 📋 Resumo e Próximos Passos

### ✅ O que você pode fazer neste notebook:

1. **Exploração Interativa**: Use o widget para testar diferentes configurações
2. **Análise de Prompts**: Compare prompts entre biomas e estações
3. **Geração de Amostras**: Teste rápido com múltiplas configurações
4. **Exportação**: Salve coleções de prompts para uso posterior

### 🔍 Insights importantes:

- Cada bioma tem características visuais únicas nos prompts
- Variações sazonais afetam cores, cobertura e iluminação
- Qualidade da pastagem influencia presença de invasoras e solo exposto
- Sistema de pós-processamento melhora realismo das imagens

### 📚 Próximos notebooks:

1. **02_Generate_Dataset.ipynb**: Geração de datasets completos
2. **03_Quality_Control.ipynb**: Análise detalhada de qualidade
3. **04_YOLO_Training.ipynb**: Treinamento de modelos YOLO

### 💡 Dicas de uso:

- Use modo rápido para testes iniciais (512px, 15 steps)
- Monitore uso de memória GPU
- Salve configurações interessantes
- Exporte prompts de qualidade para reuso
